In [5]:
# Import necessary libraries and utility functions
import numpy as np
import pandas as pd
import networkx as nx
from Setups.DynamicStochasticUtils import (
    generate_imperfect_grid_adjacency_matrix,
    generate_poisson_arrivals,
    generate_exponential_sojourn_times,
    create_riders_and_drivers,
    create_status_dataframe,
    optimal_offline_benchmark
)



In [6]:
def greedy_matching(riders, drivers):
    matched_riders = set()
    matched_drivers = set()

    # Perform greedy matching
    for t in riders.keys():
        for rider in riders[t]:
            for driver in drivers[t]:
                if driver not in matched_drivers and driver.location == rider.location and driver.patience > 0:
                    matched_riders.add(rider)
                    matched_drivers.add(driver)
                    break

    # Calculate unmatched riders and drivers
    all_riders = set(r for t_list in riders.values() for r in t_list)
    all_drivers = set(d for t_list in drivers.values() for d in t_list)
    unmatched_riders = all_riders - matched_riders
    unmatched_drivers = all_drivers - matched_drivers

    # Calculate total wait time for matched riders and drivers
    total_wait_time_riders = sum(rider.sojourn_time for rider in matched_riders)
    total_wait_time_drivers = sum(driver.sojourn_time for driver in matched_drivers)

    # Summary statistics
    total_riders = len(all_riders)
    total_drivers = len(all_drivers)
    matched_riders_count = len(matched_riders)
    unmatched_riders_count = len(unmatched_riders)
    matched_drivers_count = len(matched_drivers)
    unmatched_drivers_count = len(unmatched_drivers)
    average_wait_time_riders = total_wait_time_riders / matched_riders_count if matched_riders_count else 0
    average_wait_time_drivers = total_wait_time_drivers / matched_drivers_count if matched_drivers_count else 0

    # Print results
    print("Summary Statistics:")
    print(f"Total Riders: {total_riders}")
    print(f"Matched Riders: {matched_riders_count}")
    print(f"Unmatched Riders: {unmatched_riders_count}")
    print(f"Total Drivers: {total_drivers}")
    print(f"Matched Drivers: {matched_drivers_count}")
    print(f"Unmatched Drivers: {unmatched_drivers_count}")
    print(f"Average Wait Time for Riders: {average_wait_time_riders:.2f}")
    print(f"Average Wait Time for Drivers: {average_wait_time_drivers:.2f}")

    return matched_riders, unmatched_riders, matched_drivers, unmatched_drivers


In [7]:
def batch_matching(riders, drivers, adj_matrix, batch_window):
    G = nx.from_numpy_array(adj_matrix)
    path_lengths = dict(nx.all_pairs_dijkstra_path_length(G))

    matched_riders = set()
    matched_drivers = set()

    all_riders = []
    all_drivers = []

    # Process in batches based on the batch window
    for t in range(0, len(riders), batch_window):
        batch_riders = []
        batch_drivers = []

        # Collect all riders and drivers in the current batch window
        for batch_t in range(t, min(t + batch_window, len(riders))):
            batch_riders.extend(riders[batch_t])
            batch_drivers.extend(drivers[batch_t])

        # Find optimal matching within the current batch window
        for rider in batch_riders:
            best_match = None
            best_cost = float('inf')

            for driver in batch_drivers:
                if driver not in matched_drivers:
                    rider_pos = rider.location[1]
                    driver_pos = driver.location[1]
                    travel_cost = path_lengths[rider_pos][driver_pos]

                    if travel_cost < best_cost:
                        best_cost = travel_cost
                        best_match = driver

            if best_match:
                matched_riders.add(rider)
                matched_drivers.add(best_match)
                batch_drivers.remove(best_match)  # Remove matched driver from the batch

        all_riders.extend(batch_riders)
        all_drivers.extend(batch_drivers)

    # Calculate unmatched riders and drivers
    unmatched_riders = set(all_riders) - matched_riders
    unmatched_drivers = set(all_drivers) - matched_drivers

    # Calculate total wait time for matched riders and drivers
    total_wait_time_riders = sum(rider.sojourn_time for rider in matched_riders)
    total_wait_time_drivers = sum(driver.sojourn_time for driver in matched_drivers)

    # Summary statistics
    total_riders = len(all_riders)
    total_drivers = len(all_drivers)
    matched_riders_count = len(matched_riders)
    unmatched_riders_count = len(unmatched_riders)
    matched_drivers_count = len(matched_drivers)
    unmatched_drivers_count = len(unmatched_drivers)
    average_wait_time_riders = total_wait_time_riders / matched_riders_count if matched_riders_count else 0
    average_wait_time_drivers = total_wait_time_drivers / matched_drivers_count if matched_drivers_count else 0

    # Print results
    print("Summary Statistics:")
    print(f"Total Riders: {total_riders}")
    print(f"Matched Riders: {matched_riders_count}")
    print(f"Unmatched Riders: {unmatched_riders_count}")
    print(f"Total Drivers: {total_drivers}")
    print(f"Matched Drivers: {matched_drivers_count}")
    print(f"Unmatched Drivers: {unmatched_drivers_count}")
    print(f"Average Wait Time for Riders: {average_wait_time_riders:.2f}")
    print(f"Average Wait Time for Drivers: {average_wait_time_drivers:.2f}")

    return matched_riders, unmatched_riders, matched_drivers, unmatched_drivers


In [8]:
# Parameters
num_nodes = 10
skip_prob = 0.15
extra_edges = 0.15
length = 100
pickup_rate = 5  # Rate of rider arrivals (Poisson)
driver_rate = 5  # Rate of driver arrivals (Poisson)
sojourn_rate = 0.4
batch_window = 5  # Batch window size

# Generate city graph
adj_matrix = generate_imperfect_grid_adjacency_matrix(num_nodes, skip_prob, extra_edges)

# Generate time series data for demand and drivers
pickup_series = generate_poisson_arrivals(pickup_rate, (length, num_nodes))
drivers_series = generate_poisson_arrivals(driver_rate, (length, num_nodes))
sojourn_times = generate_exponential_sojourn_times(sojourn_rate, length)

# Assuming random dropoffs for simplicity
dropoffs = { (t, node): [np.random.randint(0, num_nodes - 1) for _ in range(pickup_series[t, node])] for t in range(length) for node in range(num_nodes)}

# Create riders and drivers
riders, drivers = create_riders_and_drivers(pickup_series, dropoffs, drivers_series, sojourn_times)

# Perform optimal offline benchmark matching
print("OPTIMAL OFFLINE BENCHMARK \n")
matched_riders_optimal, unmatched_riders_optimal, matched_drivers_optimal, unmatched_drivers_optimal = optimal_offline_benchmark(riders, drivers, adj_matrix)

# Perform greedy matching
print("\n\nGREEDY MATCHING \n")
matched_riders_greedy, unmatched_riders_greedy, matched_drivers_greedy, unmatched_drivers_greedy = greedy_matching(riders, drivers)


# Perform batch matching
print("\n\nBATCH MATCHING \n")
matched_riders_batch, unmatched_riders_batch, matched_drivers_batch, unmatched_drivers_batch = batch_matching(riders, drivers, adj_matrix, batch_window)




OPTIMAL OFFLINE BENCHMARK 

Summary Statistics:
Total Riders: 4979
Matched Riders: 4893
Unmatched Riders: 86
Total Drivers: 4893
Matched Drivers: 4893
Unmatched Drivers: 0
Average Wait Time for Riders: 2.49
Average Wait Time for Drivers: 2.53


GREEDY MATCHING 

Summary Statistics:
Total Riders: 4979
Matched Riders: 3718
Unmatched Riders: 1261
Total Drivers: 4893
Matched Drivers: 3718
Unmatched Drivers: 1175
Average Wait Time for Riders: 2.51
Average Wait Time for Drivers: 2.51


BATCH MATCHING 

Summary Statistics:
Total Riders: 4979
Matched Riders: 4777
Unmatched Riders: 202
Total Drivers: 116
Matched Drivers: 4777
Unmatched Drivers: 116
Average Wait Time for Riders: 2.54
Average Wait Time for Drivers: 2.54
